# PennyLane QGAN

In this notebook, we walk through the main features of PennyLane.

First, we import `pennylane` itself, as well as a wrapped version of `numpy` which is provided via PennyLane. We don't need `numpy` yet, but it is good practice to import this from the start. The wrapped version of `numpy` can be used pretty much the exact same way as regular `numpy`, while providing additional support for computing gradients.

In [2]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.7 MB/s eta 0:00:00


In [3]:
import pennylane as qml
from pennylane import numpy as np

We'll also need to instantiate a `device`, which indicates where we want computations to be run (e.g., on a local simulator or a remote hardware backend).

For this tutorial, we'll work with the `'default.qubit'` device provided by PennyLane — a simple pure-state qubit simulator.

In [4]:
dev = qml.device('default.qubit', wires=2)

For all devices, `device()` accepts the following arguments:

`name`: the name of the device to be loaded

`wires`: the number of subsystems to initialize the device with

### Creating quantum circuits

As a working example, let's consider the following two qubit variational circuit:

1. Begin in the state $|0\rangle\otimes|0\rangle$ (this is assumed by default in PennyLane)
2. Apply local rotations about the $x-$axis to each qubit, with rotation angles $\theta_1$ and $\theta_2$
3. Apply a CNOT gate
4. Measure the Pauli-Z observable $P_z$ on the first qubit

We create this circuit in PennyLane the following way:

In [7]:
from pennylane.ops import Hadamard, RX, CNOT

# Declare quantum circuit
@qml.qnode(dev)
def circuit(theta1, theta2):
    Hadamard(wires=1)
    RX(theta1, wires=0)
    RX(theta2, wires=1)
    CNOT(wires=[0,1])
   # return qml.expval.PauliZ(wires=0)
    return qml.expval(qml.PauliZ(0))

In [8]:
# We can evaluate the circuit at any value of the parameters
print(circuit(0, 0))
for theta1, theta2 in np.random.random((5,2)):
    print(circuit(theta1, theta2))

0.9999999999999998
0.7196111757259164
0.9793358951304635
0.9680202904491607
0.9412305749973572
0.7582377489734851


Let's walk through the different pieces in the circuit above.

1. A quantum circuit is declared using standard Python function notation (`def fn(...)`)
2. This function contains operators (e.g., gates) found in `pennylane.ops`, which specify the computation
3. The function returns one or more `expval` objects, indicating measurements which are made at the end of the circuit
4. We use the `qnode` decorator to indicate that this is not a typical Python function. This prevents the function from being run as usual by the Python interpretor. Instead, the function is evaluated on the device `dev` (which may be quantum hardware)

#### Notes about circuits:
- Final measurements are considered as *expectation values* $\langle \cdots \rangle$ -- i.e., averages -- not single-shot results. Expectation values are deterministic, whereas single-shot measurements are stochastic. This is what allows us to do machine learning on the circuit (Note: the same principle holds for deep learning models).
- Since circuits are meant to be run on quantum hardware, there is limited support for classical computation *inside* the circuit function. On the other hand, classical processing of circuit inputs/outputs is fully supported

In [10]:
# PennyLane supports a number of common gates used in variational circuits
qml.ops.qubit

<module 'pennylane.ops.qubit' from '/usr/local/lib/python3.10/dist-packages/pennylane/ops/qubit/__init__.py'>

In [12]:
# And several choices of measurement operators
qml.expval

<function pennylane.measurements.expval.expval(op: Union[pennylane.operation.Operator, pennylane.measurements.mid_measure.MeasurementValue])>

### Gradients of quantum circuits

To do quantum machine learning, we will want to use a *gradient descent* strategy. For this, we define a final objective (or cost) function $C$ which is to be optimized with respect to the free parameters $\alpha=(\theta_1, \theta_2)$.

In this case, we'll just use the expectation value itself as cost function, i.e., $C=\langle P_Z \rangle$.

The core feature of PennyLane is that it performs *automatic differentiation* of quantum circuits. This means that it can automatically compute $\nabla_\mathbf{\alpha}C$ without any intervention from the user.

Internally, PennyLane leverages the 'parameter shift' trick for computing derivatives, i.e., it evaluates derivatives as the difference of two circuit evaluations with shifted parameters:

$$C(\theta_1, \theta_2) = \texttt{circuit}(\theta_1, \theta_2)$$

$$\partial_{\theta_1} C = a\big[ \texttt{circuit}(\theta_1+s, \theta_2) - \texttt{circuit}(\theta_1 - s, \theta_2) \big]$$

The values of the shift and scale parameters $s$ and $a$ typically depend only on the *type* of gate, and not its location, i.e., they are the same no matter where the gate appears in the circuit.

By using this method, PennyLane provides a hardware-scalable way to compute gradients and to optimize quantum circuits for QML.

In [13]:
grad_circuit = qml.grad(circuit, argnum=[0,1])
# We can evaluate the circuit gradient at any value of the parameters
for theta1, theta2 in np.random.random((5,2)):
    print("Value: ({:3f},{:3f}); Gradient={}".format(theta1, theta2,
                                             np.stack(grad_circuit(theta1, theta2))))

Value: (0.855029,0.544332); Gradient=[-0.75458969  0.        ]
Value: (0.015560,0.106824); Gradient=[-0.01555956  0.        ]
Value: (0.224957,0.640655); Gradient=[-2.23064494e-01 -5.55111512e-17]
Value: (0.998239,0.603729); Gradient=[-0.84051799  0.        ]
Value: (0.280955,0.210892); Gradient=[-0.27727356  0.        ]


### Optimizing quantum circuits

Being able to compute the gradient of a variational quantum circuit with respect to any of its parameters gives us great power. This opens up the door to using the gradient descent procedure to optimize the cost function.

The gradient descent algorithm has two steps:

1. Compute the gradient $\nabla_\alpha C$
2. Update the parameters $\alpha$ in proportion to this gradient, i.e.,
$$\alpha \mapsto \alpha - \eta \nabla_\alpha C.$$

The scaling factor $\eta$ is known as the *learning rate*.

This procedure is carried out automatically by the PennyLane `GradientDescentOptimizer` object.

In [15]:
from pennylane.optimize import GradientDescentOptimizer

# Set learning rate
eta = 0.1
opt = GradientDescentOptimizer(eta)

# Initialize parameters
init_val = np.random.random(2)  # Random values for theta1 and theta2
num_steps = 10

# Perform optimization
params = init_val
for step in range(num_steps):
    params = opt.step(lambda p: circuit(p[0], p[1]), params)  # Pass both parameters
    print(f"Step {step + 1}, Value: {params}")

print("Initial value:", init_val)
print("Final value after optimization:", params)

Step 1, Value: [0.44208174 0.50858907]
Step 2, Value: [0.48486394 0.50858907]
Step 3, Value: [0.53147274 0.50858907]
Step 4, Value: [0.58215308 0.50858907]
Step 5, Value: [0.63713545 0.50858907]
Step 6, Value: [0.69662498 0.50858907]
Step 7, Value: [0.76078825 0.50858907]
Step 8, Value: [0.82973751 0.50858907]
Step 9, Value: [0.90351293 0.50858907]
Step 10, Value: [0.9820635  0.50858907]
Initial value: [0.40287524 0.50858907]
Final value after optimization: [0.9820635  0.50858907]


In [18]:
# Confirm that automatic update does what we expect
#new_val_manual = init_val - eta * np.stack(qml.grad(circuit, argnum=[0,1])(*init_val))
#assert np.allclose(new_val, new_val_manual)

new_val = opt.step(lambda p: circuit(p[0], p[1]), init_val)

# Confirm that automatic update does what we expect
# Compute gradients with respect to both parameters
grad_values = qml.grad(circuit, argnum=[0, 1])(*init_val)

# Manually compute the new value using the learning rate and gradients
new_val_manual = init_val - eta * np.array(grad_values)

# Check that the automatic update matches the manual update
assert np.allclose(new_val, new_val_manual), "The manual update does not match the automatic update!"



In [19]:
# Note: There are a number of other optimizers in the
# gradient descent family which are available in PennyLane
qml.optimize.__all__

['AdagradOptimizer',
 'AdamOptimizer',
 'AdaptiveOptimizer',
 'GradientDescentOptimizer',
 'MomentumOptimizer',
 'NesterovMomentumOptimizer',
 'RMSPropOptimizer',
 'QNGOptimizer',
 'RiemannianGradientOptimizer',
 'RotosolveOptimizer',
 'RotoselectOptimizer',
 'ShotAdaptiveOptimizer',
 'SPSAOptimizer',
 'QNSPSAOptimizer']

PennyLane allows us to string together quantum and classical computations in highly-structured ways, and the combined hybrid computation can be differentiated and trained end-to-end.

This opens up the possibility of doing many interesting things. For example:

- Pre-processing (or post-processing) the input (output) of a quantum circuit using a neural network. Both the classical and quantum components can have trainable weights!
- Combining the power of several quantum circuits, either in series or in parallel. These circuits can even be running on different hardware!
- Training a model using both GPUs and QPUs.

Here is a simple example of a classical function post-processing the output of a quantum circuit:

In [20]:
# Define a classical cost function which post-processes the circuit's output
target = 0.33
def cost(weights):
    expval = circuit(weights[0], weights[1])
    error = np.abs(expval - target) ** 2 + weights[2] ** 2
    return error

# Evaluate cost at a random starting point
weights = np.random.random(3)
cost(weights)


tensor(0.7883158, requires_grad=True)

In [21]:
# Implement gradient descent over 100 steps
for step in range(100):
    weights = opt.step(cost, weights)
    if step % 10 == 0:
        print("Step {}: Cost={}".format(step, cost(weights)))

Step 0: Cost=0.6092365601949201
Step 10: Cost=0.061263928260112926
Step 20: Cost=0.0018967166564416436
Step 30: Cost=3.966835525222087e-05
Step 40: Cost=7.863795838258335e-07
Step 50: Cost=1.5494380150925967e-08
Step 60: Cost=3.0528172026255464e-10
Step 70: Cost=6.017757137784075e-12
Step 80: Cost=1.1866435447086984e-13
Step 90: Cost=2.3404478583473076e-15


In [22]:
# Confirm we are near the optimum
# (some random initial values may take more steps to reach)
assert np.allclose(circuit(weights[0], weights[1]), target)

## Example: Quantum Generative Adversarial Networks (QGANs)

A QGAN is the quantum version of a *Generative Adversarial Network*. These are a particular kind of deep learning model used to generate real-looking synthetic data (such as images).

![GAN images](biggan.png)

### GANs
GANs have a unique structure, consisting of two models:

- the **generator**: its goal is to produce realistic-looking data samples
- the **discriminator**: its goal is distinguish fake data produced by the discriminator from real data

Training of GANs proceeds as follows:

1. "Real" data is captured in some *training dataset*.

2. The generator produces "fake" data by starting with a random input vector and transforming it into an output.

3. The discriminator is fed samples of both real and fake data and must decide which label to assign ('real' or 'fake').

4. Training consists of alternating steps: (i) the generator is frozen and the discriminator is trained to distinguish real from fake. (ii) the discriminator is frozen and the generator is trained to fool the discriminator. The gradient of the discriminator's output provides a training signal for the generator to improve its fake generated data.

5. Eventually, this training method should converge to a stage where the generator is producing realistic data and the discriminator can't tell real from fake.

**Note:** Training is done via the *gradient descent* algorithm, updating only the weights associated to the generator (or vice versa) at each step. There is no internal structure imposed on the generator or discriminator models except that they are differentiable.


![GAN structure](gan.png)

 ### Quantum GANs

This demo walks through how to build and train a simple Quantum Generative Adversarial Network (QGAN) using PennyLane.

References:
- [Lloyd and Weedbrook (2018)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.121.040502)
- [Dallaire-Demers and Killoran (2018)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324))

Like classical GANs, QGANs consist of a generator and a discriminator. *QGANs use variational quantum circuits for both generator and discriminator.*

In [23]:
# instantiate a device for the QGAN example
gan_dev = qml.device('default.qubit', wires=3)

In classical GANs, the starting point is to draw samples either from some "real data" distribution, or from the generator, and feed them to the discriminator. In this QGAN example, we will use a quantum circuit to generate the real data.

**Note:** One of the areas where QGANs can provide a clear advantage is in modelling *quanum data*, i.e., data that is created by quantum systems. Quantum data distributions should be hard for classical models to emulate efficiently.

For this simple example, our real data will be a qubit that has been rotated (from the starting state $\left|0\right\rangle$) to some arbitrary, but fixed, state.

In [24]:
def real(phi, theta, omega):
    qml.Rot(phi, theta, omega, wires=0)

For the generator and discriminator, we will choose the same basic circuit structure, but acting on different wires.

Both the real data circuit and the generator will output on wire 0, which will be connected as an input to the discriminator. Wire 1 is provided as a workspace for the generator, while the discriminator's output will be on wire 2.

In [25]:
# Note: the structure of these circuits is more-or-less chosen arbitrarily. There is
# nothing particular about these choices of generator or discriminator

def generator(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)

def discriminator(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[1,2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

We create two QNodes. One where the real data source is wired up to the discriminator, and one where the generator is connected to the discriminator. They can both be run on the same device.

In [35]:
#import numpy as np
#import pennylane as qml
import pennylane.numpy as pnp
#from pennylane.optimize import GradientDescentOptimizer

# Define your quantum device
gan_dev = qml.device('default.qubit', wires=3)  # Adjust the number of wires if needed

# Example implementations for real, generator, and discriminator
@qml.qnode(gan_dev)
def real_disc_circuit(phi, theta, omega, disc_weights):
    real(phi, theta, omega)  # Your implementation for the real state
    discriminator(disc_weights)  # Your implementation for the discriminator
    return qml.expval(qml.PauliZ(2))  # Measure PauliZ on wire 2

@qml.qnode(gan_dev)
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)  # Your implementation for the generator
    discriminator(disc_weights)  # Your implementation for the discriminator
    return qml.expval(qml.PauliZ(2))  # Measure PauliZ on wire 2

# Cost functions as provided
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true  # generator wants to minimize this prob

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost

def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

# Initialize parameters
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7

np.random.seed(0)
eps = 1e-2

# Create generator weights as trainable parameters
gen_weights = pnp.array([np.pi] + [0] * 8, requires_grad=True) + pnp.random.normal(scale=eps, size=[9])

# Create discriminator weights as trainable parameters
disc_weights = pnp.random.normal(size=[9], requires_grad=True)

# Create an optimizer
opt = GradientDescentOptimizer(0.1)

# Optimization loop for the discriminator
for it in range(50):
    disc_weights = opt.step(disc_cost, disc_weights)
    cost = disc_cost(disc_weights)
    if it % 5 == 0:
        print("Discriminator Step {}: cost = {}".format(it + 1, cost))

# Optimization loop for the generator
for it in range(50):
    gen_weights = opt.step(gen_cost, gen_weights)
    cost = gen_cost(gen_weights)
    if it % 5 == 0:
        print("Generator Step {}: cost = {}".format(it + 1, cost))


Discriminator Step 1: cost = -0.10942017805789106
Discriminator Step 6: cost = -0.38998842264903105
Discriminator Step 11: cost = -0.6660191175815624
Discriminator Step 16: cost = -0.8550839212078469
Discriminator Step 21: cost = -0.9454459581664484
Discriminator Step 26: cost = -0.9805878247866401
Discriminator Step 31: cost = -0.9931371328342748
Discriminator Step 36: cost = -0.9974896764916589
Discriminator Step 41: cost = -0.998986350663071
Discriminator Step 46: cost = -0.9995000463932009
Generator Step 1: cost = -0.00026646913829925234
Generator Step 6: cost = -0.00042662008589333666
Generator Step 11: cost = -0.0006872486146980994
Generator Step 16: cost = -0.0011111626380133077
Generator Step 21: cost = -0.0018000510248328827
Generator Step 26: cost = -0.0029179304125442895
Generator Step 31: cost = -0.004727717539773746
Generator Step 36: cost = -0.00764662888103157
Generator Step 41: cost = -0.012325866735736213
Generator Step 46: cost = -0.019754518934527177


##### Cost function

There are two ingredients to the cost here.
- The first is the probability that the discriminator correctly classifies real data as real.
- The second ingredient is the probability that the discriminator classifies fake data (i.e., a state prepared by the generator) as real.

The discriminator's objective is to maximize the probability of correctly classifying real data, while minimizing the probability of mistakenly classifying fake data.

The generator's objective is to maximize the probability that the discriminator accepts fake data as real.

##### Optimization
We initialize the fixed angles of the "real data" circuit, as well as the initial parameters for both generator and discriminator. These are chosen so that the generator initially prepares a state on wire 0 that is very close to the $\left| 1 \right\rangle$ state.

In the first stage of training, we optimize the discriminator while keeping the generator parameters fixed.

At the discriminator's optimum, the probability for the discriminator to correctly classify the real data should be close to one.

In [36]:
prob_real_true(disc_weights)

tensor(0.9998972, requires_grad=True)

For comparison, we check how the discriminator classifies the generator's (still unoptimized) fake data:

In [37]:
prob_fake_true(gen_weights, disc_weights)

tensor(0.02862489, requires_grad=True)

In the adverserial training method, we have to now train the generator to better fool the discriminator

In more complex settings, we can continue training the models in an alternating fashion until we reach the optimum point of the two-player adversarial game.

In [38]:
for it in range(200):
    gen_weights = opt.step(gen_cost, gen_weights)
    cost = -gen_cost(gen_weights)
    if it % 5 == 0:
        print("Step {}: cost = {}".format(it, cost))

Step 0: cost = 0.03136834673567196
Step 5: cost = 0.04909734599307841
Step 10: cost = 0.07520378135265493
Step 15: cost = 0.11169015288702189
Step 20: cost = 0.15917286333741015
Step 25: cost = 0.21566031343947323
Step 30: cost = 0.2763735721045274
Step 35: cost = 0.3354169186527468
Step 40: cost = 0.3883501266928636
Step 45: cost = 0.43371772120148633
Step 50: cost = 0.4728490188392828
Step 55: cost = 0.5087778323625844
Step 60: cost = 0.5451977336157676
Step 65: cost = 0.5856632916397905
Step 70: cost = 0.6327897835085232
Step 75: cost = 0.6872469221106644
Step 80: cost = 0.7468453348018459
Step 85: cost = 0.8066413637587676
Step 90: cost = 0.8607353038575852
Step 95: cost = 0.9048413990478853
Step 100: cost = 0.937668744186236
Step 105: cost = 0.9604104860258096
Step 110: cost = 0.975371147847834
Step 115: cost = 0.9848746701785991
Step 120: cost = 0.9907765008479255
Step 125: cost = 0.994389853595337
Step 130: cost = 0.9965827747752587
Step 135: cost = 0.9979065439048094
Step 140: 

At the optimum of the generator, the probability for the discriminator to be fooled should be close to 1.

In [39]:
prob_real_true(disc_weights)

tensor(0.9998972, requires_grad=True)

In [40]:
prob_fake_true(gen_weights, disc_weights)

tensor(0.99989438, requires_grad=True)

At the joint optimum the overall cost will be close to zero.

In [41]:
disc_cost(disc_weights)

tensor(-2.81796094e-06, requires_grad=True)

The generator has successfully learned how to simulate the real data enough to fool the discriminator!